In [19]:

# coding: utf-8

# In[ ]:


from __future__ import print_function

import sys
import numpy as np
from math import sqrt 
from time import time
import matplotlib.pyplot as plt 

sys.path.append('/home/xilinx')
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))

    print("Start of \"" + sys.argv[0] + "\"")

    ol = Overlay("/home/xilinx/r09525031/Scalable_Floating-Point_Matrix_Inversion/design_1_wrapper.bit")
    luinv = ol.luinv_0;
    
    size = 1024;
    golden = open("/home/xilinx/r09525031/Scalable_Floating-Point_Matrix_Inversion/hw_gold.txt", "r+");
    input_data = open("/home/xilinx/r09525031/Scalable_Floating-Point_Matrix_Inversion/hw_in.txt", "r+");
    
    in_r = allocate(shape=(size,), dtype=np.float32)
    in_i = allocate(shape=(size,), dtype=np.float32)
    out_r = allocate(shape=(size,), dtype=np.float32)
    out_i = allocate(shape=(size,), dtype=np.float32)

    for i in range(size):
        i_d = input_data.readline();
        in_r[i] = float(i_d);
        i_d = input_data.readline();
        in_i[i] = float(i_d);
    
    
    timeKernelStart = time()

    luinv.write(0x18, in_r.device_address)
    luinv.write(0x20, in_i.device_address)
    luinv.write(0x28, out_r.device_address)
    luinv.write(0x30, out_i.device_address)
    luinv.write(0x00, 0x01)
    while (luinv.read(0x00) & 0x4) == 0x0:
        continue
    timeKernelEnd = time()
    print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s")
      
    print("Data check")
    error_r = 0;
    error_i = 0;
    
    for i in range(size):
        g_d = golden.readline();
        gold_r = float(g_d);
        g_d = golden.readline();
        gold_i = float(g_d);
        if(gold_r != out_r[i]):
            error_r += 1;
        if(gold_i != out_i[i]):
            error_i += 1;
        
                  
    print("Error num for R:" + str(error_r));
    print("Error num for I:" + str(error_i));
    
    golden.close();
    input_data.close();
    print("============================")
    print("Exit process")

Entry: /usr/lib/python3/dist-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/lib/python3/dist-packages/ipykernel_launcher.py"
Kernel execution time: 0.0022356510162353516 s
Data check
Error num for R:0
Error num for I:0
Exit process
